# Process MIMIC-TPW data
- add time coordinate by extracting from filename
- concatenate into single dataset
- add lat/lon coordinates
- restrict to tropical Atlantic
- re-save reduced single dataset into new netcdf file

In [1]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import rc, colors, ticker
import matplotlib.dates as mdates
from scipy.interpolate import interp2d, RectBivariateSpline
from datetime import datetime, timedelta
import pandas as pd
import cftime
import seaborn as sns
import json
from matplotlib import rc
import glob
import os
from thermo_functions import *
from time_series_diag import *
from bowtie_read_data import *

In [ ]:
# Load MIMIC-TPW data
file_paths08 = glob.glob("/huracan/tank1/work/ORCESTRA/mimic-tpw/202408/*.nc")
file_paths09 = glob.glob("/huracan/tank1/work/ORCESTRA/mimic-tpw/202409/*.nc")

file_paths = file_paths08 + file_paths09

#Parse the filename to extract the date and time information
times= [pd.to_datetime(os.path.basename(fp)[4:12] + os.path.basename(fp)[13:15], format='%Y%m%d%H') for fp in file_paths]

#Open the files and assign time coordinates
datasets = [xr.open_dataset(fp).assign_coords(time=time) for fp, time in zip(file_paths, times)]

#Combine into a single dataset and fix coordinates
ds = xr.concat(datasets, dim='time')
ds_reordered = ds.sortby('time')
lonDim = ds_reordered.isel(time=0).lonArr.values
latDim = ds_reordered.isel(time=0).latArr.values
ds_reordered = ds_reordered.assign_coords(lon=lonDim)
ds_reordered = ds_reordered.assign_coords(lat=latDim)
mimic = ds_reordered.sel(lat=slice(-10,50), lon=slice(-90,60))
mimic_reduced = mimic.drop_vars(['tpwGridPrior','tpwGridSubseq','timeAwayGridPrior','timeAwayGridSubseq','footGridPrior','footGridSubseq','satGridPrior','satGridSubseq'])

#Write out to new netcdf file
mimic_reduced.to_netcdf('/huracan/tank1/work/ORCESTRA/mimic-tpw/mimic_tpw_20240809.nc')